# SVM e ANN

## Introdução

Na cidade de Nova Iorque, a estimação da quantidade de ciclistas faz parte do planejamento do sistema de transporte público urbano. Neste contexto a quantidade de ciclistas passando pelas pontes do East River são contadas diariamente.
Essa base de dados foi disponibilizada pela prefeitura de Nova Iorque e foi limpada para ser postada no Kaggle.

Nesse contexto a base de dados [New York City - East River Bicycle Crossings](https://www.kaggle.com/new-york-city/nyc-east-river-bicycle-crossings) permite uma análise da quantidade de ciclistas que passam pela ponte.



## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, RandomizedSearchCV, StratifiedKFold
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

## East River Bicycle Crossings
### Read

In [2]:
X = pd.read_csv('nyc-east-river-bicycle-counts.csv', index_col=0)
display(X.head())
print(f'Dataframe shape: {X.shape}')

,Date,Day,High Temp (°F),Low Temp (°F),Precipitation,Brooklyn Bridge,Manhattan Bridge,Williamsburg Bridge,Queensboro Bridge,Total
0,2016-04-01 00:00:00,2016-04-01 00:00:00,78.1,66.0,0.01,1704.0,3126,4115.0,2552.0,11497
1,2016-04-02 00:00:00,2016-04-02 00:00:00,55.0,48.9,0.15,827.0,1646,2565.0,1884.0,6922
2,2016-04-03 00:00:00,2016-04-03 00:00:00,39.9,34.0,0.09,526.0,1232,1695.0,1306.0,4759
3,2016-04-04 00:00:00,2016-04-04 00:00:00,44.1,33.1,0.47 (S),521.0,1067,1440.0,1307.0,4335
4,2016-04-05 00:00:00,2016-04-05 00:00:00,42.1,26.1,0,1416.0,2617,3081.0,2357.0,9471


Dataframe shape: (210, 10)


### Feature Engineering


In [3]:
X['Precipitation'] = np.where(X['Precipitation'] == '0.47 (S)', '0.47', X['Precipitation'])
X.drop(X[X.Precipitation == 'T'].index,inplace=True)
y = X[['Brooklyn Bridge', 'Manhattan Bridge', 'Williamsburg Bridge','Queensboro Bridge', 'Total']]
X.drop(['Brooklyn Bridge', 'Manhattan Bridge', 'Williamsburg Bridge','Queensboro Bridge', 'Total','Day','Date'], inplace=True, axis=1)
print(f'Features data shape: {X.shape}')
print(f'Target data shape: {y.shape}')

Features data shape: (203, 3)
Target data shape: (203, 5)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train.head()

,High Temp (°F),Low Temp (°F),Precipitation
12,57.0,39.9,0
156,57.0,53.1,0.09
28,57.0,46.9,0.05
84,66.9,54.0,0
149,64.0,48.0,0


In [6]:
# Create model
for i in y_train.columns:
    svr = SVR()
    svr.fit(X_train, y_train[i])

    print(i+' Training scores')
    y_hat = svr.predict(X_train)
    mae = mean_absolute_error(y_train[i], y_hat)
    print('mae:', mae)
    mse = mean_squared_error(y_train[i], y_hat)
    print('mse:', mse)
    r2 = r2_score(y_train[i], y_hat)
    print('R2:', r2)

    print(' Test scores')
    y_pred = svr.predict(X_test)
    mae = mean_absolute_error(y_test[i], y_pred)
    print('mae:', mae)
    mse = mean_squared_error(y_test[i], y_pred)
    print('mse:', mse)
    r2 = r2_score(y_test[i], y_pred)
    print('R2:', r2,"\n")

Brooklyn Bridge Training scores
mae: 837.0018271492429
mse: 964568.8591974631
R2: 0.002753737636042586
 Test scores
mae: 825.1076469036552
mse: 1029491.3985823691
R2: -0.05484156955482322 

Manhattan Bridge Training scores
mae: 1459.3431476854996
mse: 2974371.7859095815
R2: 0.005240438762876276
 Test scores
mae: 1384.6223710714291
mse: 2926940.043950223
R2: -0.05904957688729073 

Williamsburg Bridge Training scores
mae: 1512.7734320875143
mse: 3310750.774057435
R2: -0.000155752038129231
 Test scores
mae: 1531.205207879265
mse: 3571937.180246939
R2: -0.0800357972688388 

Queensboro Bridge Training scores
mae: 938.7857777665268
mse: 1200216.092061242
R2: 0.010731748021516219
 Test scores
mae: 938.034476171948
mse: 1299116.4909557593
R2: -0.010300912155091657 

Total Training scores
mae: 4761.483308630724
mse: 31419912.81791413
R2: -0.0007372070109119555
 Test scores
mae: 4697.86374446463
mse: 33070664.49172937
R2: -0.06315437105050892 



In [7]:
# Criação de um grid aleatorizado
tuned_parameters =[
    {'kernel': ['rbf'],
     'gamma': [1e-2, 1e-1, 1],
     'C': [1, 10, 100, 1000]
    },
    {'kernel': ['linear'], 
     'C': [1, 10, 100, 1000]
    },
    {'kernel': ['poly'],
     'degree': [2,3,4],
     'C': [1, 10, 100, 1000]
    }
]

In [8]:
lista={}
for i,j in zip(y_train.columns, range(0, len(y_train.columns))):
    scores = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

    score = scores[0]
    print("# Ajuste dos hyper-parâmetros (%s)" % score)
    print()
    
    svr_cv = RandomizedSearchCV(
        SVR(), tuned_parameters, cv=5, scoring=score, refit=True
    )
    lista[j]=svr_cv
    print(i,j,"\n",svr_cv.fit(X_train, y_train[i]))

# Ajuste dos hyper-parâmetros (neg_mean_squared_error)

Brooklyn Bridge 0 
 RandomizedSearchCV(cv=5, error_score=nan,
                   estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                                 epsilon=0.1, gamma='scale', kernel='rbf',
                                 max_iter=-1, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='deprecated', n_iter=10, n_jobs=None,
                   param_distributions=[{'C': [1, 10, 100, 1000],
                                         'gamma': [0.01, 0.1, 1],
                                         'kernel': ['rbf']},
                                        {'C': [1, 10, 100, 1000],
                                         'kernel': ['linear']},
                                        {'C': [1, 10, 100, 1000],
                                         'degree': [2, 3, 4],
                                         'kernel': ['poly']}],
                   pre_dispatch='

In [9]:
for i in range(0,len(lista)):
    print(lista[i].best_params_)

{'kernel': 'linear', 'C': 1000}
{'kernel': 'linear', 'C': 10}
{'kernel': 'rbf', 'gamma': 1, 'C': 1000}
{'kernel': 'rbf', 'gamma': 0.1, 'C': 1000}
{'kernel': 'linear', 'C': 1000}


In [10]:
for i,j in zip(y_train.columns, range(0, len(y_train.columns))):
    print(i,"\n",'Training scores')
    y_hat = lista[j].predict(X_train)
    mae = mean_absolute_error(y_train[i], y_hat)
    print('mae:', mae)
    mse = mean_squared_error(y_train[i], y_hat)
    print('mse:', mse)
    r2 = r2_score(y_train[i], y_hat)
    print('R2:', r2)

    print('Test scores')
    y_pred = lista[j].predict(X_test)
    mae = mean_absolute_error(y_test[i], y_pred)
    print('mae:', mae)
    mse = mean_squared_error(y_test[i], y_pred)
    print('mse:', mse)
    r2 = r2_score(y_test[i], y_pred)
    print('R2:', r2)
    print(i,j)
    print("\n")

Brooklyn Bridge 
 Training scores
mae: 471.2547954290519
mse: 380118.66582141543
R2: 0.6070037767333951
Test scores
mae: 344.44011429679955
mse: 178907.81104554178
R2: 0.8166867673991569
Brooklyn Bridge 0


Manhattan Bridge 
 Training scores
mae: 963.6660055761893
mse: 1413854.2723730314
R2: 0.5271455094142073
Test scores
mae: 888.6078333733941
mse: 1056444.1252259265
R2: 0.6177486771079863
Manhattan Bridge 1


Williamsburg Bridge 
 Training scores
mae: 7.033473969397888
mse: 2597.3543182730045
R2: 0.9992153565644816
Test scores
mae: 36.62759972489505
mse: 13683.580072741626
R2: 0.9958625374502432
Williamsburg Bridge 2


Queensboro Bridge 
 Training scores
mae: 44.885344237390285
mse: 29376.902907230062
R2: 0.9757863291620538
Test scores
mae: 76.60818446156563
mse: 47391.0239283341
R2: 0.9631448026130935
Queensboro Bridge 3


Total 
 Training scores
mae: 2865.1717669448417
mse: 12862469.407034772
R2: 0.5903250341827713
Test scores
mae: 2625.9735526621034
mse: 9898304.65127032
R2: 0.681

In [11]:
# Create model
annr = MLPRegressor(max_iter=10000, learning_rate_init=0.01)
annr.fit(X_train, y_train)

print('Training scores')
y_hat = annr.predict(X_train)
mae = mean_absolute_error(y_train, y_hat)
print('mae:', mae)
mse = mean_squared_error(y_train, y_hat)
print('mse:', mse)
r2 = r2_score(y_train, y_hat)
print('R2:', r2)

print('Test scores')
y_pred = annr.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mae:', mae)
mse = mean_squared_error(y_test, y_pred)
print('mse:', mse)
r2 = r2_score(y_test, y_pred)
print('R2:', r2)

Training scores
mae: 1508.0638161202708
mse: 5116743.721818784
R2: 0.35450505655045284
Test scores
mae: 1239.0637742440788
mse: 3707581.006350218
R2: 0.5262470892302462


In [12]:
# Criação de um grid aleatorizado
tuned_parameters =[
    {
        'solver': ['adam'],
        'hidden_layer_sizes': [1, 10, 100, 1000],
        'activation': ['relu', 'identity', 'logistic', 'tanh'],
        'learning_rate_init': [1e-3, 1e-2, 1e-1, 1]
    },
     {
        'solver': ['lbfgs'],
        'hidden_layer_sizes': [1, 10, 100, 1000],
        'activation': ['relu', 'identity', 'logistic', 'tanh'],
    }
]

In [13]:
scores = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

score = scores[0]
print("# Ajuste dos hyper-parâmetros (%s)" % score)
print()

annr_cv = RandomizedSearchCV(
    MLPRegressor(max_iter=10000), tuned_parameters, cv=10, scoring=score, refit=True
)
annr_cv.fit(X_train, y_train)

# Ajuste dos hyper-parâmetros (neg_mean_squared_error)



G:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
G:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
G:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
G:\ANACONDA\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (10000) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)
G:\ANACONDA\lib\site-package

RandomizedSearchCV(cv=10, error_score=nan,
                   estimator=MLPRegressor(activation='relu', alpha=0.0001,
                                          batch_size='auto', beta_1=0.9,
                                          beta_2=0.999, early_stopping=False,
                                          epsilon=1e-08,
                                          hidden_layer_sizes=(100,),
                                          learning_rate='constant',
                                          learning_rate_init=0.001,
                                          max_fun=15000, max_iter=10000,
                                          momentum=0.9, n_iter_no_change=10,
                                          nesterovs_momentum=True, power_t=0.5,
                                          rand...
                   param_distributions=[{'activation': ['relu', 'identity',
                                                        'logistic', 'tanh'],
                                    

In [14]:
print(annr_cv.best_params_)

{'solver': 'lbfgs', 'hidden_layer_sizes': 100, 'activation': 'tanh'}


In [15]:
print('Training scores')
y_hat = annr_cv.predict(X_train)
mae = mean_absolute_error(y_train, y_hat)
print('mae:', mae)
mse = mean_squared_error(y_train, y_hat)
print('mse:', mse)
r2 = r2_score(y_train, y_hat)
print('R2:', r2)

print('Test scores')
y_pred = annr_cv.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print('mae:', mae)
mse = mean_squared_error(y_test, y_pred)
print('mse:', mse)
r2 = r2_score(y_test, y_pred)
print('R2:', r2)

Training scores
mae: 1136.360875996174
mse: 3710278.225853692
R2: 0.5329005481634793
Test scores
mae: 1274.4574683417059
mse: 4195933.967073979
R2: 0.462178733373862
